In [18]:
from kubernetes import client, config
import json
import os
from influxdb import InfluxDBClient

In [ ]:
"""
Grafana
http://192.168.1.120:3000/
id : admin
pwd : paas4ml
"""

In [19]:
try:
    host = '192.168.1.120'
    port = '8086'
    name = 'paas4ml'
    pwd = 'paas4ml'
    db = 'k8s-metric'
    influxdb_mgr = InfluxDBClient(host=host, port=port, username=name, password=pwd, database=db)
except Exception as exp:
    print(str(exp))

In [20]:
config.load_kube_config()
api = client.CustomObjectsApi()

In [38]:
k8s_nodes = api.list_cluster_custom_object("metrics.k8s.io", "v1beta1", "nodes")
fields = {'cpu':'', 'memory':''}
for stats in k8s_nodes['items']:
    table_name = 'k8s-cluster-resource-%s' %(stats['metadata']['name'])
    fields['cpu'] = int(stats['usage']['cpu'][:-1])
    fields['memory'] = int(stats['usage']['memory'][:-2])
    influx_json = [{
                    'measurement': table_name,
                    'fields': fields
                }]
    try:
        if influxdb_mgr.write_points(influx_json) is True:
            print('influx write success:' + str(influx_json))
        else:
            print('influx write faile:' + str(influx_json))
    except Exception as exp:
        print(str(exp)) 

influx write success:[{'measurement': 'k8s-cluster-resource-master1', 'fields': {'cpu': 136230638, 'memory': 1799820}}]
influx write success:[{'measurement': 'k8s-cluster-resource-worker1', 'fields': {'cpu': 26071171, 'memory': 560700}}]
influx write success:[{'measurement': 'k8s-cluster-resource-worker2', 'fields': {'cpu': 38866556, 'memory': 1710740}}]
influx write success:[{'measurement': 'k8s-cluster-resource-worker3', 'fields': {'cpu': 55593116, 'memory': 1090400}}]


In [2]:
kubeconfig = os.getenv('KUBECONFIG')
config.load_kube_config(kubeconfig)
v1 = client.CoreV1Api()

In [3]:
api_client = client.ApiClient()

In [4]:
ret_metrics = api_client.call_api('/apis/metrics.k8s.io/v1beta1/namespaces/default/pods', 'GET', auth_settings = ['BearerToken'], response_type='json', _preload_content=False) 

In [5]:
response = ret_metrics[0].data.decode('utf-8')

In [42]:
metric_info = json.loads(response)
display(metric_info)

{'kind': 'PodMetricsList',
 'apiVersion': 'metrics.k8s.io/v1beta1',
 'metadata': {},
 'items': [{'metadata': {'name': 'p4ml-grafana-7ccd5db9fc-48rrt',
    'namespace': 'default',
    'creationTimestamp': '2023-08-21T12:21:25Z',
    'labels': {'app': 'p4ml-grafana', 'pod-template-hash': '7ccd5db9fc'}},
   'timestamp': '2023-08-21T12:21:02Z',
   'window': '16.989s',
   'containers': [{'name': 'p4ml-grafana',
     'usage': {'cpu': '492664n', 'memory': '29432Ki'}}]},
  {'metadata': {'name': 'p4ml-influxdb-5fbbf46b5c-clnbh',
    'namespace': 'default',
    'creationTimestamp': '2023-08-21T12:21:25Z',
    'labels': {'app': 'p4ml-influxdb', 'pod-template-hash': '5fbbf46b5c'}},
   'timestamp': '2023-08-21T12:21:14Z',
   'window': '28.651s',
   'containers': [{'name': 'p4ml-influxdb',
     'usage': {'cpu': '751779n', 'memory': '88808Ki'}}]},
  {'metadata': {'name': 'p4ml-jupyter-7ff768476-ffwh7',
    'namespace': 'default',
    'creationTimestamp': '2023-08-21T12:21:25Z',
    'labels': {'app': 

In [53]:

for idx, pod_id in enumerate(metric_info['items']):
    name = metric_info['items'][idx]['containers'][0]['name']
    cpu_n = int(metric_info['items'][idx]['containers'][0]['usage']['cpu'][:-1])
    memory_n = int(metric_info['items'][idx]['containers'][0]['usage']['memory'][:-2])
    # print(name, cpu_n, memory_n)
    fields = {'cpu':'', 'memory':''}
    fields['cpu'] = cpu_n
    fields['memory'] = memory_n
    table_name = 'k8s-pod-resource-%s' %(name)
    influx_json = [{
                    'measurement': table_name,
                    'fields': fields
                }]
    try:
        if influxdb_mgr.write_points(influx_json) is True:
            print('influx write success:' + str(influx_json))
        else:
            print('influx write faile:' + str(influx_json))
    except Exception as exp:
        print(str(exp)) 

influx write success:[{'measurement': 'k8s-pod-resource-p4ml-grafana', 'fields': {'cpu': 492664, 'memory': 29432}}]
influx write success:[{'measurement': 'k8s-pod-resource-p4ml-influxdb', 'fields': {'cpu': 751779, 'memory': 88808}}]
influx write success:[{'measurement': 'k8s-pod-resource-p4ml-jupyter', 'fields': {'cpu': 633664, 'memory': 944556}}]
influx write success:[{'measurement': 'k8s-pod-resource-p4ml-minio', 'fields': {'cpu': 348468, 'memory': 105232}}]
influx write success:[{'measurement': 'k8s-pod-resource-p4ml-mongodb', 'fields': {'cpu': 4091747, 'memory': 186580}}]
influx write success:[{'measurement': 'k8s-pod-resource-p4ml-postgres', 'fields': {'cpu': 40125, 'memory': 29664}}]
